# Cours LangChain TP5

## Use case

Ce TP est inspiré du cas d'utilisation de compréhension du [code émis dans la bibliothèque LangChain](https://colab.research.google.com/github/langchain-ai/langchain/blob/master/docs/docs/use_cases/code_understanding.ipynb). Il vous permettra d'utiliser LangChain pour analyser du code et en générer

L'analyse du code source est l'une des applications LLM les plus populaires (par exemple, GitHub Copilot, Code Interpreter, Codium et Codeium) pour des cas d'utilisation tels que :

* Q&R sur la base de code pour comprendre comment elle fonctionne
* Utilisation des LLM pour suggérer des refactors ou des améliorations
* Utilisation des LLM pour documenter le code

![Image description](https://raw.githubusercontent.com/langchain-ai/langchain/master/docs/static/img/code_understanding.png)


## Vue d'ensemble

Le pipeline pour l'assurance qualité sur le code suit les étapes que nous suivons pour répondre aux questions sur les documents, avec quelques différences :

En particulier, nous pouvons employer une stratégie de splitting qui fait plusieurs choses :

* Chaque fonction et classe de haut niveau du code est chargée dans des documents distincts.
* Le code source est chargé dans des documents distincts.
* Conserve les métadonnées sur l'origine de chaque fractionnement.

## QuickStart

In [ ]:
#!pip install --upgrade --quiet  langchain-openai tiktoken chromadb langchain gitpython
# Set env var OPENAI_API_KEY or load from a .env file

import os
os.environ['OPENAI_API_KEY']="OPENAI_API_KEY" # À Modifier

Nous suivrons la structure de [ce notebook](https://github.com/cristobalcl/LearningLangChain/blob/master/notebooks/04%20-%20QA%20with%20code.ipynb) et utiliserons le [context aware code splitting](https://python.langchain.com/docs/integrations/document_loaders/source_code).

In [ ]:
from git import Repo
from langchain_community.document_loaders.generic import GenericLoader
from langchain_community.document_loaders.parsers import LanguageParser
from langchain_text_splitters import Language

# Clone
repo_path = "test_repo/"
if not(os.path.isdir(repo_path)):
    repo = Repo.clone_from("https://github.com/langchain-ai/langchain", to_path=repo_path)

On charge le code Python en utilisant [`LanguageParser`](https://python.langchain.com/docs/integrations/document_loaders/source_code), qui va:

* Conserver les fonctions et les classes de haut niveau ensemble (dans un seul document)
* Mettre le reste du code dans un document séparé
* Conserve les métadonnées sur l'origine de chaque split

In [ ]:
# Load
loader = GenericLoader.from_filesystem(
    repo_path + "/libs/langchain/langchain",
    glob="**/*",
    suffixes=[".py"],
    exclude=["**/non-utf8-encoding.py"],
    parser=LanguageParser(language=Language.PYTHON, parser_threshold=500),
)
documents = loader.load()
len(documents)

1562

### Splitting

On fractionnne le `Document` en morceaux (chunks) pour les tranformer en embedding et les stocker en vectorDB

Nous pouvons utiliser `RecursiveCharacterTextSplitter` avec `language` spécifié.

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

python_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.PYTHON, chunk_size=1200, chunk_overlap=100
)
texts = python_splitter.split_documents(documents)
len(texts)

3151

### RetrievalQ&R

Nous devons stocker les documents de manière à pouvoir effectuer des recherches sémantiques sur leur contenu.

L'approche la plus courante consiste à intégrer le contenu de chaque document, puis à stocker l'intégration et le document dans un magasin vectoriel.

Lors de la configuration de l'extracteur vectoriel :

* Nous testons [max marginal relevance](/docs/use_cases/question_answering) pour la recherche.
* Et 8 documents sont retournés

In [ ]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

db = Chroma.from_documents(texts, OpenAIEmbeddings(model="text-embedding-3-small",disallowed_special=()))
retriever = db.as_retriever(
    search_type="mmr",  # Also test "similarity"
    search_kwargs={"k": 8},
)

In [ ]:
db.search("import",search_type="mmr")

[Document(page_content='# Code for: def _import_openllm() -> Any:\n\n\n# Code for: def _import_openlm() -> Any:\n\n\n# Code for: def _import_pai_eas_endpoint() -> Any:\n\n\n# Code for: def _import_petals() -> Any:\n\n\n# Code for: def _import_pipelineai() -> Any:\n\n\n# Code for: def _import_predibase() -> Any:\n\n\n# Code for: def _import_predictionguard() -> Any:\n\n\n# Code for: def _import_promptlayer() -> Any:\n\n\n# Code for: def _import_promptlayer_chat() -> Any:\n\n\n# Code for: def _import_replicate() -> Any:\n\n\n# Code for: def _import_rwkv() -> Any:\n\n\n# Code for: def _import_sagemaker_endpoint() -> Any:\n\n\n# Code for: def _import_self_hosted() -> Any:\n\n\n# Code for: def _import_self_hosted_hugging_face() -> Any:\n\n\n# Code for: def _import_stochasticai() -> Any:\n\n\n# Code for: def _import_symblai_nebula() -> Any:\n\n\n# Code for: def _import_textgen() -> Any:\n\n\n# Code for: def _import_titan_takeoff() -> Any:\n\n\n# Code for: def _import_titan_takeoff_pro() -> A

### Chat

In [ ]:
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationSummaryMemory
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
memory = ConversationSummaryMemory(
    llm=llm, memory_key="chat_history", return_messages=True
)
qr = ConversationalRetrievalChain.from_llm(llm, retriever=retriever, memory=memory)

In [ ]:
question = "Comment initialiser un Agent Python avec LangChain?"
result = qr.invoke(question)
result["answer"]

'Pour initialiser un Agent Python avec LangChain, vous pouvez suivre les étapes suivantes :\n\n1. Importez les outils nécessaires :\n\n    ```python\n    from langchain_core.agents import AgentExecutor, create_json_chat_agent\n    from langchain_community.chat_models import ChatOpenAI\n    from langchain import hub\n    ```\n\n2. Récupérez le modèle de prompt à partir du hub de LangChain :\n\n    ```python\n    prompt = hub.pull("nom_du_modele")\n    ```\n\n3. Créez une instance du modèle que vous souhaitez utiliser pour l\'Agent :\n\n    ```python\n    model = ChatOpenAI()\n    ```\n\n4. Créez l\'Agent en utilisant le modèle, les outils et le prompt :\n\n    ```python\n    agent = create_json_chat_agent(model, tools, prompt)\n    ```\n\n5. Créez un exécuteur d\'Agent et invoquez l\'Agent avec une entrée :\n\n    ```python\n    agent_executor = AgentExecutor(agent=agent, tools=tools)\n    agent_executor.invoke({"input": "votre_message_ici"})\n    ```\n\nAssurez-vous d\'adapter les noms

In [ ]:
#qr.memory.clear()